# Lesson 1B: Logistic Regression PyTorch Practical

## Introduction

In Lesson 1A, we explored logistic regression theory and coded from scratch a simple logistic regression model to classify breast cancer samples.

Now we'll implement the same model using PyTorch, one of the most popular deep learning frameworks, to create a practical breast cancer classifier.

This lesson focuses on implementation by:

1. Building an efficient PyTorch-based logistic regression model
2. Working with real medical data from the Wisconsin Breast Cancer dataset
3. Learning industry-standard code organization patterns
4. Establishing good practices for model development and evaluation

## Table of Contents



## Required Libraries

Before we get started, let's load the necessary libraries.

In this lesson we will use the following libraries:

| Library | Purpose |
|---------|---------|
| Pandas | Data tables and data manipulation |
| Numpy | Numerical computing functions|
| PyTorch | Deep learning framework |
| Matplotlib | Plotting functions |
| Seaborn | Statistical visualisation |
| Scikit-learn | Machine learning utilities including logistic regression, preprocessing, metrics, and dataset loading functions |
| Typing | Type hints |

In [ ]:
# Standard library imports
from typing import List, Optional, Union, Tuple, Dict, Any
import json
from datetime import datetime
import logging
import hashlib


# Third party imports - core data science
import numpy as np
import pandas as pd
from numpy.typing import NDArray

# PyTorch imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn utilities
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score, 
    recall_score,
    f1_score,
    confusion_matrix,
    roc_curve, 
    roc_auc_score,
    auc
)

# Jupyter specific configuration
%matplotlib inline

# Configuration settings for our libraries
np.random.seed(42)
torch.manual_seed(42)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Configure logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")

### Why PyTorch for Logistic Regression?

While we built logistic regression from scratch in Lesson 1A, PyTorch offers several key advantages:

1. **Efficient Computation**
   - Automatic differentiation
   - GPU acceleration when available
   - Optimized numerical operations

2. **Production-Ready Tools**
   - Built-in data loading utilities
   - Memory-efficient batch processing
   - Robust optimization algorithms

3. **Reusable Patterns**
   - Model organization with `nn.Module`
   - Data handling with `Dataset` and `DataLoader`
   - Training loops and evaluation workflows

These fundamentals will serve us well throughout our machine learning journey, particularly when we move on to neural networks (Lesson 3). 

This is because our PyTorch logistic regression implementation is technically speaking a single layer neural network and we will be using the same techniques to build more complex neural networks.

### What We'll Build

We'll create a complete cancer diagnosis system that:
- Processes the Wisconsin Breast Cancer dataset
- Implements logistic regression in PyTorch
- Trains efficiently using mini-batches
- Evaluates performance on real medical data

By the end of this lesson, you'll have both a working cancer classifier and practical experience with PyTorch development.

Let's begin by setting up our development environment with the necessary libraries...

## The Wisconsin Breast Cancer Dataset:

When doctors examine breast tissue samples under a microscope, they look for specific cellular characteristics that might indicate cancer:

1. **Cell Size and Shape**
   - Radius (mean distance from center to perimeter)
   - Perimeter (size of the outer boundary)
   - Area (total space occupied by the cell)
   - Cancer cells often appear larger and more irregular

2. **Texture Analysis**
   - Surface variations and patterns
   - Standard deviation of gray-scale values
   - Malignant cells typically show more variation

3. **Cell Boundaries**
   - Compactness (perimeter² / area)
   - Concavity (severity of concave portions)
   - Cancer cells often have irregular, ragged boundaries

### Dataset Structure

The dataset contains 569 samples with confirmed diagnoses. For each biopsy sample, we have:
- 30 numeric features capturing the aforementioned cell characteristics
- Binary classification: Malignant (1) or Benign (0)

This presents an ideal scenario for logistic regression because:
1. Clear binary outcome (malignant vs benign)
2. Numeric features that can be combined linearly
3. Well-documented medical relationships
4. Real-world impact of predictions

Our task mirrors a real diagnostic challenge: Can we use these cellular measurements to predict whether a tumor is cancerous? This is exactly the kind of high-stakes binary classification problem where logistic regression's interpretable predictions become crucial - doctors need to understand not just what the model predicts, but how confident it is in that prediction.

## Loading and exploring the dataset

Let's explore the Wisconsin Breast Cancer dataset through a series of visualizations and analyses to understand our data better. Let's start by:
 
   1. Getting a basic overview of our dataset
      - Look at the first few rows of each feature in a table format
      - Check how many samples and features we have
      - Display summary statistics for each feature (mean, std, min, max, skewness, kurtosis)
      
   2. Investigating the distribution of our features
      - Generate box plots for each feature (30 plots), categorized by diagnosis to compare measurements between cancerous and non-cancerous cases
      - Generate histograms with kernel density estimation (KDE) overlays (30 plots) to visualize the shape and spread of each feature's distribution

   3. Investigating relationships between features
      - Create three sets of paired plots for the most distinct pairs
      - Create three sets of paired plots for the least distinct pairs
      - Create three sets of paired plots for moderately distinct pairs
      (Total of 15 scatter plots arranged in a 5x3 grid)

   4. Examining correlations
      - Analyzing how each feature correlates with the diagnosis of cancer
      - Investigating how features correlate with one another
      - Utilizing these findings to guide our selection of features


In [ ]:
def load_cancer_data():
   """Load and prepare breast cancer dataset."""
   cancer = load_breast_cancer()
   df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
   df['target'] = cancer.target
   return df

def plot_initial_analysis(df):
   """Plot comprehensive initial data analysis including skewness and kurtosis."""
   # Print basic information
   print("=== Dataset Overview ===")
   display(df.head())
   print(f"\nShape: {df.shape}")
   
   print("\n=== Summary Statistics ===")
   stats = pd.DataFrame({
       'mean': df.mean(),
       'std': df.std(),
       'min': df.min(),
       'max': df.max(),
       'skew': df.skew(),
       'kurtosis': df.kurtosis()
   }).round(3)
   display(stats)
   
   # Box plots for each feature by diagnosis
   n_features = len(df.columns) - 1  # Excluding target column
   n_rows = (n_features + 4) // 5
   
   fig, axes = plt.subplots(n_rows, 5, figsize=(20, 4*n_rows))
   axes = axes.ravel()
   
   tumor_colors = {1: '#4CAF50', 0: '#FF4B4B'}
   
   for idx, feature in enumerate(df.columns[:-1]):
       plot_df = pd.DataFrame({
           'value': df[feature],
           'diagnosis': df['target'].map({0: 'Malignant', 1: 'Benign'})
       })
       
       sns.boxplot(data=plot_df, x='diagnosis', y='value', 
                  hue='diagnosis', palette=[tumor_colors[0], tumor_colors[1]],
                  legend=False, ax=axes[idx])
       axes[idx].set_title(f'{feature}\nSkew: {df[feature].skew():.2f}\nKurt: {df[feature].kurtosis():.2f}')
       axes[idx].set_xlabel('')
       
       if max(plot_df['value']) > 1000:
           axes[idx].tick_params(axis='y', rotation=45)
   
   for idx in range(n_features, len(axes)):
       axes[idx].set_visible(False)
   
   plt.suptitle('Feature Distributions by Diagnosis', y=1.02, size=16)
   plt.tight_layout()
   plt.show()
   
   # Distribution plots (5 per row)
   n_rows = (n_features + 4) // 5
   fig, axes = plt.subplots(n_rows, 5, figsize=(20, 4*n_rows))
   axes = axes.ravel()
   
   for idx, feature in enumerate(df.columns[:-1]):
       sns.histplot(df[feature], ax=axes[idx], kde=True)
       axes[idx].set_title(f'{feature}\nSkew: {df[feature].skew():.2f}\nKurt: {df[feature].kurtosis():.2f}')
       
   for idx in range(n_features, len(axes)):
       axes[idx].set_visible(False)
       
   plt.suptitle('Feature Distributions', y=1.02, size=16)
   plt.tight_layout()
   plt.show()

def plot_feature_pairs(df):
    """Plot selected informative feature pairs in a 3x3 or 3x5 grid."""
    # Get feature correlations with target
    target_corr = df.corr()['target'].abs().sort_values(ascending=False)
    
    # Get feature pair correlations
    corr_matrix = df.iloc[:, :-1].corr().abs()
    
    # 1. Top 5 most separating pairs (highest correlation with target)
    top_features = target_corr[1:6].index
    top_pairs = [(f1, f2) for i, f1 in enumerate(top_features) 
                 for j, f2 in enumerate(top_features[i+1:], i+1)][:5]
    
    # 2. 5 pairs with minimal separation
    # Get features with low target correlation
    low_corr_features = target_corr[target_corr < 0.3].index
    low_sep_pairs = [(f1, f2) for i, f1 in enumerate(low_corr_features) 
                     for j, f2 in enumerate(low_corr_features[i+1:], i+1)][:5]
    
    # 3. 5 interesting pairs showing partial separation
    # Features with moderate target correlation
    mod_corr_features = target_corr[(target_corr >= 0.3) & (target_corr < 0.6)].index
    mod_sep_pairs = [(f1, f2) for i, f1 in enumerate(mod_corr_features) 
                     for j, f2 in enumerate(mod_corr_features[i+1:], i+1)][:5]
    
    # Combine all pairs
    all_pairs = top_pairs + low_sep_pairs + mod_sep_pairs
    
    # Plot pairs
    fig, axes = plt.subplots(3, 5, figsize=(20, 12))
    axes = axes.ravel()
    
    tumor_colors = {1: '#4CAF50', 0: '#FF4B4B'}
    
    for idx, (feat1, feat2) in enumerate(all_pairs):
        sns.scatterplot(data=df, x=feat1, y=feat2, hue='target',
                       palette=tumor_colors, ax=axes[idx], alpha=0.6)
        corr_val = corr_matrix.loc[feat1, feat2]
        target_corr1 = target_corr[feat1]
        target_corr2 = target_corr[feat2]
        
        title = f'Correlation: {corr_val:.2f}\nTarget corr: {target_corr1:.2f}, {target_corr2:.2f}'
        axes[idx].set_title(title)
        axes[idx].set_xlabel(feat1, rotation=45)
        axes[idx].set_ylabel(feat2, rotation=45)
        axes[idx].tick_params(axis='both', labelsize=8)
        if idx >= 10:  # Only show legend on last row
            axes[idx].legend(title='Diagnosis')
        else:
            axes[idx].legend().remove()
    
    plt.suptitle('Feature Pair Relationships\nTop: Best Separation | Middle: Poor Separation | Bottom: Partial Separation', 
                y=1.02, size=16)
    plt.tight_layout()
    plt.show()

# Execute analysis
df = load_cancer_data()
plot_initial_analysis(df)
plot_feature_pairs(df)

## Exploratory Data Analysis

Building on our implementation from Lesson 1A, let's analyze the Wisconsin Breast Cancer dataset to inform our PyTorch approach.

### Dataset Overview

The dataset contains 569 breast tissue biopsies with confirmed diagnoses:
```python
# Class Distribution
Benign:    357 (62.7%)  # Non-cancerous samples
Malignant: 212 (37.3%)  # Cancerous samples
```

Each biopsy has 30 measurements that capture cell characteristics, making this an ideal case for logistic regression as demonstrated in Lesson 1A.

### Key Data Characteristics

1. **Feature Scale Variations**
   ```python
   # Primary measurements show wide scale differences
   radius:     14.127 ± 3.524   # Base cell measurements
   area:      654.889 ± 351.914 # Derived measurements
   smoothness:  0.096 ± 0.014   # Texture measurements
   
   # Range spans multiple orders of magnitude
   area:        143.5 - 2501.0  # Will need standardization
   radius:        6.9 - 28.1    # As in Lesson 1A
   smoothness:    0.05 - 0.16   # Smallest scale features
   ```

2. **Distribution Patterns**
   ```python
   # Feature distributions by skewness
   Normal:       smoothness (0.46), texture (0.50)  # Linear relationships
   Right-skewed: radius (0.94), area (1.65)        # Size features
   Heavy-tailed: perimeter error (3.44)            # Diagnostic signals
   
   # Error terms remain important (from Lesson 1A)
   perimeter error: 2.866 ± 2.022  # Outliers indicate malignancy
   area error:     40.337 ± 45.491 # Keep these variations
   ```

3. **Feature-Target Relationships**
   ```python
   # Strong linear correlations (validated in 1A)
   worst concave points: -0.794  # Key diagnostic feature
   worst perimeter:      -0.783  # Size indicator
   mean concave points:  -0.777  # Shape characteristic
   
   # Multiple strong predictors suggest
   Top 5 features: r = -0.794 to -0.743  # Linear model suitable
   ```

### Implementation Evolution: From NumPy to PyTorch

Our EDA findings inform how we'll upgrade our Lesson 1A implementation:

1. **Data Processing Improvements**
   ```python
   # Scale differences (handled in Lesson 1A by)
   area:        143.5 - 2501.0  # 4 orders of magnitude
   smoothness:    0.05 - 0.16   # <1 order of magnitude
   → Replace standardise_features() with StandardScaler
   → Replace manual array handling with DataLoader
   
   # Class imbalance (managed in 1A by)
   Benign:    357 (62.7%)  # Majority class
   Malignant: 212 (37.3%)  # Minority class
   → Replace train_test_split_with_stratification() with stratified DataLoader
   → Add weighted metrics evaluation
   ```

2. **Model Architecture Upgrades**
   ```python
   # Linear relationships (from 1A functions)
   Feature correlations: -0.794 to -0.743
   → Replace calculate_linear_scores() with nn.Linear
   → Replace convert_scores_to_probabilities() with nn.Sigmoid
   → Add Xavier initialization
   
   # Probability needs (from 1A methods)
   Class split: 62.7% vs 37.3%
   → Replace calculate_probabilities() with forward()
   → Replace manual BCE with nn.BCELoss
   → Add probability calibration checks
   ```

3. **Training Enhancements**
   ```python
   # Optimization challenges (from 1A functions)
   Scale range: 0.05 to 2501.0
   Convergence: Required manual tuning
   → Replace train_model() with PyTorch training loop
   → Replace manual gradient descent with Adam
   → Add early stopping mechanism
   
   # Prediction improvements
   Clinical needs: Precision, recall crucial
   → Replace predict_binary_classes() with torch.where()
   → Expand beyond accuracy_score
   → Add ROC curve analysis
   ```

### Next Steps

Building on our Lesson 1A implementation, we'll upgrade each component using PyTorch:

1. **Enhanced Data Pipeline**
   - Convert standardise_features() to StandardScaler
   - Replace train_test_split_with_stratification() with DataLoader
   - Maintain stratified sampling approach

2. **Modernized Model Architecture**
   - Convert calculate_linear_scores() to nn.Linear
   - Replace probability calculations with PyTorch functions
   - Add proper initialization methods

3. **Robust Training Process**
   - Upgrade train_model() to PyTorch training loop
   - Implement early stopping
   - Add comprehensive metrics

These improvements will maintain the mathematical foundations from Lesson 1A while leveraging PyTorch's optimized implementations and additional features.

## Implementing a PyTorch Logistic Regression for Cancer Diagnosis

Building on our theoretical understanding from Lesson 1A, let's implement a logistic regression model using PyTorch, one of the most popular deep learning frameworks. 

This modern implementation introduces several powerful features and optimizations while maintaining the same core mathematical principles we learned previously.

In [ ]:
def prepare_data(df: pd.DataFrame) -> Tuple[NDArray, NDArray, NDArray, NDArray, StandardScaler]:
    """Prepare data for PyTorch model training by splitting and scaling.
    
    This function follows the same preprocessing steps from our numpy implementation
    in Lesson 1A, but prepares data specifically for PyTorch:
    1. Separates features and target
    2. Creates stratified train/test split
    3. Standardizes features using training data statistics
    
    Args:
        df: DataFrame containing cancer measurements and diagnosis
            Features should be numeric measurements (e.g., cell size, shape)
            Target should be binary (0=benign, 1=malignant)
    
    Returns:
        Tuple containing:
        - X_train_scaled: Standardized training features
        - X_test_scaled: Standardized test features
        - y_train: Training labels
        - y_test: Test labels
        - scaler: Fitted StandardScaler for future use
    """
    # Separate features and target
    X = df.drop('target', axis=1).values  # Features as numpy array
    y = df['target'].values               # Labels as numpy array

    # Split data - using same 80/20 ratio and stratification as Lesson 1A
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2,           # 20% test set
        random_state=42,         # For reproducibility
        stratify=y               # Maintain class balance
    )
    
    # Scale features using training data statistics
    # Note: We standardize error terms without normalizing distribution
    # because their skewness might indicate malignancy
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

class CancerDataset(Dataset):
    """PyTorch Dataset wrapper for cancer data.
    
    This class bridges our numpy arrays from prepare_data() to PyTorch's
    efficient data loading system. It:
    1. Converts numpy arrays to PyTorch tensors
    2. Provides length information for batch creation
    3. Enables indexed access for efficient mini-batch sampling
    
    Args:
        X: Feature array (standardized measurements)
        y: Label array (0=benign, 1=malignant)
    """
    def __init__(self, X: NDArray, y: NDArray):
        # Convert numpy arrays to PyTorch tensors with appropriate types
        self.X = torch.FloatTensor(X)            # Features as 32-bit float
        self.y = torch.FloatTensor(y).reshape(-1, 1)  # Labels as 2D tensor
        
    def __len__(self) -> int:
        """Return dataset size for batch calculations."""
        return len(self.X)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """Enable indexing for batch sampling."""
        return self.X[idx], self.y[idx]

class CancerClassifier(nn.Module):
    """PyTorch binary classifier for cancer diagnosis.
    
    This implements the same logistic regression model from Lesson 1A, but using
    PyTorch's neural network framework. Key components:
    1. Linear layer: Computes weighted sum (z = wx + b)
    2. Sigmoid activation: Converts sum to probability
    3. Xavier initialization: For stable training with standardized features
    
    Args:
        input_features: Number of measurements used for diagnosis
    """
    def __init__(self, input_features: int):
        super().__init__()
        # Single linear layer - matches our numpy implementation
        self.linear = nn.Linear(input_features, 1)
        # Sigmoid activation - same as Lesson 1A
        self.sigmoid = nn.Sigmoid()
        
        # Initialize weights using Xavier/Glorot initialization
        # This ensures good starting point with standardized features
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Compute diagnosis probability.
        
        This exactly mirrors our numpy implementation:
        1. Linear combination of features
        2. Sigmoid activation for probability
        """
        return self.sigmoid(self.linear(x))
    
    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """Convert probabilities to binary predictions.
        
        Args:
            x: Input features as tensor
            
        Returns:
            Binary predictions (0=benign, 1=malignant)
        """
        with torch.no_grad():  # No gradient tracking needed
            probabilities = self(x)
            # Default threshold of 0.5 - same as Lesson 1A
            return (probabilities > 0.5).float()

def train_model(
    model: CancerClassifier, 
    train_loader: DataLoader, 
    val_loader: DataLoader,
    epochs: int = 1000,
    lr: float = 0.001,
    patience: int = 5
) -> Tuple[CancerClassifier, Dict]:
    """Train cancer classifier with early stopping.
    
    This implements the same training process as Lesson 1A but with PyTorch's:
    1. Automatic differentiation for gradients
    2. Mini-batch processing for efficiency
    3. Adam optimizer for adaptive learning rates
    4. Early stopping to prevent overfitting
    
    Args:
        model: PyTorch cancer classifier
        train_loader: DataLoader for training batches
        val_loader: DataLoader for validation batches
        epochs: Maximum training iterations
        lr: Learning rate for optimization
        patience: Epochs to wait before early stopping
        
    Returns:
        Tuple of (trained model, training history)
    """
    # Binary Cross Entropy - same loss as Lesson 1A
    criterion = nn.BCELoss()
    # Adam optimizer - handles feature scale differences well
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Early stopping setup
    best_val_loss = float('inf')
    best_weights = None
    no_improve = 0
    
    # Training history for visualization
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': []
    }
    
    for epoch in range(epochs):
        # Training phase
        model.train()  # Enable gradient tracking
        train_losses = []
        train_correct = 0
        train_total = 0
        
        for X_batch, y_batch in train_loader:
            # Forward pass - get diagnosis probabilities
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            
            # Backward pass - learn feature importance
            optimizer.zero_grad()  # Clear previous gradients
            loss.backward()        # Compute gradients
            optimizer.step()       # Update weights
            
            # Track metrics
            train_losses.append(loss.item())
            train_correct += ((y_pred > 0.5) == y_batch).sum().item()
            train_total += len(y_batch)
        
        # Validation phase
        model.eval()  # Disable gradient tracking
        val_losses = []
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = model(X_batch)
                val_losses.append(criterion(y_pred, y_batch).item())
                val_correct += ((y_pred > 0.5) == y_batch).sum().item()
                val_total += len(y_batch)
        
        # Calculate epoch metrics
        train_loss = sum(train_losses) / len(train_losses)
        val_loss = sum(val_losses) / len(val_losses)
        train_acc = train_correct / train_total
        val_acc = val_correct / val_total
        
        # Store history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}')
            print(f'Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}\n')
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = model.state_dict().copy()
            no_improve = 0
        else:
            no_improve += 1
            if no_improve == patience:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    # Restore best weights
    model.load_state_dict(best_weights)
    return model, history

def plot_training_curves(history: Dict[str, List[float]]) -> None:
    """Visualize training progression.
    
    Creates side-by-side plots of:
    1. Loss curves - Shows learning progression
    2. Accuracy curves - Shows diagnostic performance
    
    Args:
        history: Dict containing training metrics
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    ax1.plot(history['train_loss'], label='Train')
    ax1.plot(history['val_loss'], label='Validation')
    ax1.set_title('Loss Over Time')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Binary Cross Entropy')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy curves
    ax2.plot(history['train_acc'], label='Train')
    ax2.plot(history['val_acc'], label='Validation')
    ax2.set_title('Accuracy Over Time')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Load and prepare data
df = load_cancer_data()
X_train_scaled, X_test_scaled, y_train, y_test, scaler = prepare_data(df)

# Create data loaders with reasonable batch size for medical data
batch_size = 32  # Small enough for precise updates, large enough for efficiency
train_dataset = CancerDataset(X_train_scaled, y_train)
val_dataset = CancerDataset(X_test_scaled, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize and train model
model = CancerClassifier(input_features=X_train_scaled.shape[1])
model, history = train_model(model, train_loader, val_loader)

# Plot training results to understand learning process
plot_training_curves(history)

# Print final metrics
with torch.no_grad():
    train_preds = model(torch.FloatTensor(X_train_scaled))
    test_preds = model(torch.FloatTensor(X_test_scaled))
    
    train_acc = ((train_preds > 0.5).float().numpy().flatten() == y_train).mean()
    test_acc = ((test_preds > 0.5).float().numpy().flatten() == y_test).mean()
    
    print(f"Final Training Accuracy: {train_acc:.4f}")
    print(f"Final Testing Accuracy: {test_acc:.4f}")

Above is a complete working PyTorch implementation, which achieves remarkable results on the Wisconsin Breast Cancer dataset - 97.8% training accuracy and 96.5% test accuracy, converging in just 447 epochs. 

This is a significant improvement over our SimpleLogisticRegression NumPy implementation from lesson 1a, both in terms of training speed and final performance.

We'll analyse the result of this model later in the lesson but first review the implementation.

Before diving deep into how each function works, let's highlight the key differences between this implementation and our Lesson 1A version:

- **Automatic Differentiation:** Instead of manually calculating gradients, PyTorch handles all gradient computation automatically through its autograd system

- **Mini-batch Processing:** Rather than processing all 455 training samples at once, we used batches of 32 samples for better memory efficiency and training dynamics 

- **Optimized Data Loading:** New CancerDataset class enables efficient data handling and GPU acceleration

- **Advanced Optimization:** Replaced simple gradient descent with Adam optimizer for adaptive learning rates

- **Early Stopping:** Added automatic training termination when validation performance stops improving

- **Production Features:** nn.Module provides proper model persistence, data validation, and performance monitoring

- **GPU Support:** Our implementation is ready for hardware acceleration without code changes

- **Industry Patterns:** We've followed PyTorch's standard model organization using nn.Module

## Understanding Our PyTorch Implementation

In Lesson 1A, we built logistic regression from scratch to understand the core mathematics. Here, we've reimplemented that same model using PyTorch's optimized framework.

While the mathematical foundations remain unchanged, our implementation organizes the code into production-ready components.

### The Core Mathematics
Our model still follows the same three mathematical steps as Lesson 1A:
1. Linear combination of inputs: z = wx + b
2. Sigmoid activation: σ(z) = 1/(1 + e^(-z))
3. Binary cross-entropy loss: -(y log(p) + (1-y)log(1-p))
4. Backward pass: Compute gradients for each parameter, determine the amount to update each parameter by, and update the weights for the next epoch

### Implementation Structure 

1. **Data Pipeline**

   The data pipeline starts with standardization - scaling cell measurements to zero mean and unit variance, just like Lesson 1A. The key difference is how we handle this standardized data. 
   
   Rather than keeping it as numpy arrays, we convert to PyTorch tensors - optimized data structures that track computations for automatic differentiation. 
   
   The DataLoader then efficiently samples these tensors in mini-batches of 32, enabling GPU acceleration and reducing memory usage.

   ```python
   # Step 1: Prepare and standardize data
   prepare_data()                                 # Returns numpy arrays
   cancer_dataset = CancerDataset(X, y)           # Converts to PyTorch tensors
   train_loader = DataLoader(            
       cancer_dataset, batch_size=32              # Creates mini-batches
   )  
   ```

2. **Model Architecture**
   
   Our CancerClassifier inherits from PyTorch's nn.Module, giving us automatic gradient computation. The __init__ method defines our learnable parameters - a linear layer for wx + b and sigmoid activation for converting to probabilities. 
   
   The xavier_uniform initialization helps ensure stable training with standardized inputs by keeping layer outputs similarly scaled. The forward method defines how data flows through these layers, and predict handles binary classification decisions.

   ```python
   class CancerClassifier(nn.Module):
       def __init__(self, input_features):         # Constructor
           self.linear = nn.Linear(30, 1)          # wx + b layer
           self.sigmoid = nn.Sigmoid()             # Activation
           nn.init.xavier_uniform_(self.weight)    # Initialize weights

       def forward(self, x):                       # Forward pass
           return self.sigmoid(self.linear(x))     # Compute probability
           
       def predict(self, x):                       # Get diagnosis
           return (self.forward(x) > 0.5).float()  # Convert to 0/1
   ``` 

3. **Training Loop**

   The training process introduces several modern optimization techniques:
   - BCELoss replaces our manual loss calculation, handling numerical stability internally
   - Adam optimizer adapts learning rates for each parameter based on gradient history
   - Early stopping monitors validation loss, stopping when no improvement for 'patience' epochs
   - Mini-batch processing enables more frequent weight updates and better generalization

   ```python
   def train_model(model, train_loader, val_loader, epochs=1000, patience=5):
       criterion = nn.BCELoss()                    # Binary Cross-Entropy
       optimizer = optim.Adam(model.parameters())  # Adaptive learning
       
       for epoch in range(epochs):
           for X_batch, y_batch in train_loader:   # Process 32 samples
               y_pred = model(X_batch)             # Forward pass
               loss = criterion(y_pred, y_batch)   # Compute error
               
               optimizer.zero_grad()               # Clear gradients
               loss.backward()                     # Backward pass
               optimizer.step()                    # Update weights
           
           if early_stopping_triggered():          # Check progress
               break                               # Stop if no improvement
   ```

4. **Performance Monitoring**

   Throughout training, we track both loss and accuracy metrics on training and validation sets. This helps us understand:
   - If the model is learning effectively (decreasing loss)
   - If it's overfitting (validation metrics getting worse)
   - When to stop training (early stopping decisions)
   - Final model performance (test set evaluation)

   ```python
   # Track training progress and results
   history = {
       'train_loss': [], 'val_loss': [],           # Loss tracking
       'train_acc': [], 'val_acc': []              # Accuracy tracking
   }
   
   plot_training_curves(history)                   # Visualize learning
   ```

In the following sections, we'll examine each component in detail:
- How tensors improve upon numpy arrays for neural computation
- Why xavier initialization helps with standardized inputs
- How Adam optimization adapts learning rates automatically
- What BCELoss and early stopping tell us about model reliability

Then we'll analyze our training results to understand how we achieved 97.8% accuracy in cancer detection.

## The Data Pipeline

In Lesson 1A, we manually prepared our cancer data step by step, handwriting each function. 

Now let's see how PyTorch and SciKit-Learn help us build a more robust pipeline. Our data journey has three key stages: preparing the data, converting it to PyTorch's format, and setting up efficient loading.

### Stage 1: Data Preparation

First, we load and prepare our medical data:

```python
df = load_cancer_data()  # Load the Wisconsin breast cancer dataset
```

Our dataset contains cell measurements and their diagnoses. But before we can use them, we need to:

1. **Separate Features from Target**
   ```python
   X = df.drop('target', axis=1).values  # All cell measurements
   y = df['target'].values               # Cancer diagnosis (0 or 1)
   ```
   This gives us two arrays: one containing all 30 cell measurements (like radius, texture, perimeter), and another containing the diagnosis (benign or malignant).

2. **Create Training and Test Sets**
   ```python
   X_train, X_test, y_train, y_test = train_test_split(
       X, y,
       test_size=0.2,          # Keep 20% for testing
       stratify=y,             # Maintain cancer/healthy ratio
       random_state=42         # For reproducibility
   )
   ```
   We're keeping 20% of our data completely separate for final testing. The `stratify=y` parameter ensures our test set has the same proportion of cancer cases as our training set - critical for medical applications.

3. **Standardize the Measurements**
   ```python
   scaler = StandardScaler()
   X_train_scaled = scaler.fit_transform(X_train)  # Learn scaling from training data
   X_test_scaled = scaler.transform(X_test)        # Apply same scaling to test data
   ```
   Just like in Lesson 1A, we standardize each feature to have mean=0 and standard deviation=1. But we only compute these statistics from the training data to avoid information leakage.

### Stage 2: PyTorch Dataset Creation

Now we wrap our prepared data in PyTorch's Dataset format:

```python
class CancerDataset(Dataset):
    def __init__(self, X: NDArray, y: NDArray):
        self.X = torch.FloatTensor(X)                # Convert features to tensor
        self.y = torch.FloatTensor(y).reshape(-1, 1) # Convert labels to 2D tensor
        
    def __len__(self):
        return len(self.X)  # Total number of samples
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]  # Get one sample and label
```

This class does three important things:
1. Converts our numpy arrays to PyTorch tensors (PyTorch's native format)
2. Reshapes the data appropriately (-1 means "figure out the right size")
3. Provides methods for accessing individual samples

We create two datasets:
```python
train_dataset = CancerDataset(X_train_scaled, y_train)  # For training
val_dataset = CancerDataset(X_test_scaled, y_test)      # For validation
```

### What's a Tensor?

Before we move on to data loading, let's understand what happened when we converted our numpy arrays to tensors:

```python
self.X = torch.FloatTensor(X)  # Converting features to tensor
```

A tensor is fundamentally similar to a numpy array - it's a container for numbers that can be arranged in different dimensions:
- A 0D tensor is a single number: `tensor(3.14)`
- A 1D tensor is like a list: `tensor([1.2, 0.5, 3.1])`
- A 2D tensor is like a table: `tensor([[1.2, 0.5], [0.8, 1.5]])`

But tensors have two special powers that make them perfect for neural networks:

1. **Automatic Gradient Tracking**
   ```python
   x = torch.tensor([1.0], requires_grad=True)
   y = x * 2  # y now remembers it came from x
   z = y ** 2 # z remembers the whole computation chain
   ```
   When we compute the gradient during training, tensors automatically track how changes should flow backward through the computations. In Lesson 1A, we had to derive and implement these gradients manually!

2. **GPU Acceleration**
   ```python
   if torch.cuda.is_available():
       x = x.cuda()  # Move to GPU
   ```
   Tensors can easily be moved to a GPU for parallel processing. Our numpy arrays in Lesson 1A could only use the CPU.

In our cancer detection pipeline, we use 2D tensors:
```python
# Feature tensor shape: [num_samples, num_features]
X_tensor = torch.FloatTensor([
    [15.2, 14.7, 98.2, ...],  # First cell's measurements
    [12.3, 11.8, 78.1, ...],  # Second cell's measurements
    # ... more cells
])

# Label tensor shape: [num_samples, 1]
y_tensor = torch.FloatTensor([
    [1],  # First diagnosis
    [0],  # Second diagnosis
    # ... more diagnoses
])
```

The `FloatTensor` part means we're using 32-bit precision - generally the best balance of accuracy and speed for machine learning. Now that our data is in tensor form, we can move on to setting up efficient loading.


### Stage 3: Data Loading

Finally, we set up efficient data loading:

```python
train_loader = DataLoader(
    train_dataset,
    batch_size=32,     # Process 32 samples at once
    shuffle=True       # Randomize order each epoch
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32
)
```

The DataLoader is like a smart iterator that:
1. Automatically creates batches of 32 samples
2. Shuffles the training data each epoch
3. Handles the memory management for us

Why 32 samples per batch? It's a sweet spot:
- Large enough to give stable gradient estimates
- Small enough to fit easily in memory
- Works well with modern GPU architectures

Now we can efficiently iterate through our data during training:
```python
for features, labels in train_loader:  # Get next batch
    # features shape: [32, 30]  (32 samples, 30 measurements each)
    # labels shape: [32, 1]     (32 diagnoses)
    # Train on this batch
```

This pipeline sets us up for efficient training by:
1. Properly separating training and test data
2. Standardizing our measurements appropriately
3. Converting data to PyTorch's optimized formats
4. Enabling efficient batch processing

In the next section, we'll see how our model uses this carefully prepared data to learn cancer diagnosis patterns.

## The CancerClassifier: From Mathematical Principles to PyTorch Implementation

In Lesson 1A, we built logistic regression from scratch using numpy, carefully deriving each mathematical component. Now we'll translate this same mathematical foundation into PyTorch's framework, understanding how each piece maps to our previous implementation while gaining powerful new capabilities.

### The Mathematical Foundation

Let's recall our core logistic regression equations from Lesson 1A:

For a single cell sample with 30 measurements x₁, x₂, ..., x₃₀, our model:
1. Computes a weighted sum: z = w₁x₁ + w₂x₂ + ... + w₃₀x₃₀ + b
2. Converts to probability: p = 1/(1 + e^(-z))
3. Makes a diagnosis: ŷ = 1 if p > 0.5 else 0

Our PyTorch implementation preserves this exact mathematical structure while adding modern optimization capabilities:

```python
class CancerClassifier(nn.Module):
    def __init__(self, input_features: int):
        super().__init__()
        self.linear = nn.Linear(input_features, 1)
        self.sigmoid = nn.Sigmoid()
        
        # Initialize weights optimally
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        z = self.linear(x)     # Weighted sum
        p = self.sigmoid(z)    # Convert to probability
        return p

    def predict(self, x):
        with torch.no_grad():
            p = self(x)
            return (p > 0.5).float()
```

### Understanding nn.Module: The Foundation

The first key difference from our numpy implementation is inheritance from nn.Module:

```python
class CancerClassifier(nn.Module):
    def __init__(self, input_features: int):
        super().__init__()
```

This inheritance provides three crucial capabilities:
1. Parameter Management: Automatically tracks all learnable parameters (weights and biases)
2. GPU Support: Can move entire model to GPU with single command
3. Gradient Computation: Enables automatic differentiation through the model

When we call super().__init__(), we're setting up this infrastructure. Think of nn.Module as providing a laboratory full of sophisticated equipment, whereas in Lesson 1A we had to build everything by hand.

### The Linear Layer: Modern Matrix Operations

In Lesson 1A, we explicitly created weight and bias arrays:
```python
# Lesson 1A approach:
self.weights = np.random.randn(input_features) * 0.01
self.bias = 0.0

def compute_weighted_sum(self, x):
    return np.dot(x, self.weights) + self.bias
```

PyTorch's nn.Linear encapsulates this same computation:
```python
# PyTorch approach:
self.linear = nn.Linear(input_features, 1)
```

But there's much more happening under the hood. The linear layer:
1. Creates a weight matrix of shape [1, input_features]
2. Creates a bias vector of shape [1]
3. Implements optimal memory layouts for matrix operations
4. Tracks gradients for both weights and bias
5. Supports batched computations automatically

For our cancer detection task with 30 features, this means:
```python
weights shape: [1, 30]  # One weight per cell measurement
bias shape: [1]        # Single bias term
```

### Weight Initialization: From Random to Principled

In Lesson 1A, we used simple random initialization:
```python
weights = np.random.randn(input_features) * 0.01
```

Our PyTorch implementation uses Xavier initialization:
```python
nn.init.xavier_uniform_(self.linear.weight)
nn.init.zeros_(self.linear.bias)
```

The mathematics behind Xavier initialization comes from analyzing the variance of activations. For a layer with nin inputs and nout outputs:

```python
# Desired variance after linear transformation
std = sqrt(2.0 / (nin + nout))

# For our case:
nin = 30  # cell measurements
nout = 1  # cancer probability
std = sqrt(2.0 / 31) ≈ 0.25

# Weights uniformly distributed in [-0.25, 0.25]
```

This initialization ensures:
1. Signal propagates well forward (preventing vanishing activations)
2. Gradients propagate well backward (preventing vanishing gradients)
3. Initial predictions are neither too confident nor too uncertain

### The Forward Pass: Computing Cancer Probability

The forward method defines our computational graph:
```python
def forward(self, x):
    z = self.linear(x)     # Step 1: Linear combination
    p = self.sigmoid(z)    # Step 2: Probability conversion
    return p
```

When processing a single cell's measurements:
```python
# Example standardized measurements
x = tensor([
    1.2,   # Radius: 1.2 standard deviations above mean
    -0.3,  # Texture: 0.3 standard deviations below mean
    1.8,   # Perimeter: 1.8 standard deviations above mean
    # ... 27 more measurements
])

# Step 1: Linear combination
z = w₁(1.2) + w₂(-0.3) + w₃(1.8) + ... + b

# Step 2: Sigmoid conversion
p = 1/(1 + e^(-z))
```

PyTorch's autograd system tracks all these computations, building a graph for backpropagation. Each operation remembers:
1. What inputs it received
2. How to compute gradients for those inputs
3. Which operations used its outputs

### The Prediction Interface: Clinical Decisions

Finally, we provide a clean interface for making diagnoses:
```python
def predict(self, x):
    with torch.no_grad():  # No need for gradients during prediction
        p = self(x)
        return (p > 0.5).float()
```

The with torch.no_grad() context:
1. Disables gradient tracking
2. Reduces memory usage
3. Speeds up computation

For a batch of cells:
```python
# Input: 32 cell samples, each with 30 measurements
X_batch shape: [32, 30]

# Output: 32 binary predictions
predictions shape: [32, 1]
values: tensor([[0.], [1.], [0.], ...])  # 0=benign, 1=malignant
```

### End-to-End Example: A Single Cell's Journey

Let's follow a single cell sample through our model:

```python
# 1. Input: Standardized cell measurements
x = tensor([
    1.2,   # Radius (high)
    -0.3,  # Texture (normal)
    1.8,   # Perimeter (very high)
    0.5,   # Area (moderately high)
    # ... 26 more measurements
])

# 2. Linear Layer: Combine evidence
z = self.linear(x)
  = 1.2w₁ - 0.3w₂ + 1.8w₃ + 0.5w₄ + ... + b
  = 2.45  # Example weighted sum

# 3. Sigmoid: Convert to probability
p = self.sigmoid(z)
  = 1/(1 + e^(-2.45))
  = 0.92  # 92% chance of cancer

# 4. Prediction: Make diagnosis
diagnosis = self.predict(x)
         = (0.92 > 0.5).float()
         = 1  # Model predicts cancer
```

Our PyTorch implementation maintains the clear mathematical reasoning of Lesson 1A while adding powerful capabilities:
1. Automatic differentiation for learning
2. Efficient batch processing
3. GPU acceleration
4. Optimal initialization
5. Memory-efficient computation

In the next section, we'll explore how this classifier learns from medical data using mini-batch processing and adaptive optimization.

## Understanding Training: How Models Learn From Data

Before diving into our train_model function's code, let's understand the fundamental concept of batch processing in machine learning. There are three main ways models can learn from data:

### Full Batch Gradient Descent (Like Our Numpy Version)

Remember our Lesson 1A implementation? It processed all training data at once:

```python
# Simple numpy version (full batch)
for epoch in range(num_epochs):
    # Calculate predictions for ALL training samples
    predictions = self.calculate_probabilities(all_features)  # All 455 samples
    
    # Calculate average error across ALL samples
    average_error = np.mean(predictions - true_labels)  # Average of 455 errors
    
    # Update weights ONCE using this average
    self.weights -= learning_rate * average_error
```

Think of this like a teacher waiting until every student (455 of them) takes a test, calculating the class average, and only then adjusting their teaching method. This is:
- Most accurate (uses all data)
- Most memory intensive (needs all data at once)
- Slowest to react (only updates once per epoch)

### Mini-Batch Gradient Descent (Our PyTorch Version)

Our current train_model function processes data in small groups:

```python
# PyTorch version (mini-batch)
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:  # Each batch has 32 samples
        # Calculate predictions for JUST THIS BATCH
        predictions = model(X_batch)  # Only 32 samples
        
        # Calculate average error for THIS BATCH
        loss = criterion(predictions, y_batch)  # Average of 32 errors
        
        # Update weights after EACH BATCH
        optimizer.step()  # Updates multiple times per epoch
```

This is like a teacher giving quizzes to groups of 32 students and adjusting their teaching after each group's results. This approach:
- Balances accuracy and speed
- Uses less memory
- Updates weights more frequently

### Stochastic Gradient Descent 

An alternative approach processes one sample at a time:

```python
# Stochastic version (not used in our code)
for epoch in range(epochs):
    for single_sample, single_label in samples:  # One at a time
        # Calculate prediction for ONE sample
        prediction = model(single_sample)  # Just 1 sample
        
        # Calculate error for THIS SAMPLE
        loss = criterion(prediction, single_label)  # Just 1 error
        
        # Update weights after EVERY sample
        optimizer.step()  # Updates very frequently
```

Like a teacher adjusting their method after each individual student's answer. This:
- Uses minimal memory
- Updates very frequently
- Can be very noisy (bounces around a lot)

### Why We Use Mini-Batches

For our cancer detection task, we chose mini-batch processing (32 samples) because:

1. Memory Efficiency
   - Processes 32 samples instead of all 455
   - Perfect for modern GPU hardware
   - Still uses vectorized operations

2. Learning Benefits
   - Updates weights more frequently than full batch
   - More stable than stochastic (single sample)
   - Good balance of speed and stability

3. Production Ready
   - Standard industry practice
   - Scales well to larger datasets
   - Works well with PyTorch's optimizations

This is what is meant by improved training dynamics - the ability to process data in smaller, more manageable chunks, allowing for more frequent weight updates and better generalization.

In the next section, we'll examine how our train_model function implements mini-batch processing step by step.

## Inside the Training Loop: Processing Mini-Batches

Now that we understand why we're using mini-batches, let's examine how our train_model function processes them. Each epoch involves processing all our training data, just in smaller chunks:

### The Training Setup

```python
criterion = nn.BCELoss()   # Same loss function as Lesson 1A
optimizer = optim.Adam(model.parameters(), lr=lr)  # We'll explain Adam later
```

The criterion (loss function) is the same binary cross-entropy we used in Lesson 1A:
```python
# What BCELoss calculates (in simple terms):
loss = -(y * log(p) + (1-y) * log(1-p))
```

### Processing One Mini-Batch

Let's follow how we process 32 cell samples:

1. **Get a Batch of Data**
   ```python
   for X_batch, y_batch in train_loader:
       # X_batch: 32 cells, 30 measurements each
       # Shape: [32, 30] like this:
       [
           [1.2, 0.8, 1.5, ...],  # First cell's measurements
           [0.5, 1.1, 0.7, ...],  # Second cell's measurements
           # ... 30 more cells
       ]

       # y_batch: 32 diagnoses (0=benign, 1=malignant)
       # Shape: [32, 1] like this:
       [
           [1],  # First cell: malignant
           [0],  # Second cell: benign
           # ... 30 more diagnoses
       ]
   ```

2. **Make Predictions**
   ```python
   y_pred = model(X_batch)  # Get predicted probabilities
   # Shape: [32, 1] with values between 0 and 1
   # Like: [[0.92], [0.15], ...] (32 predictions)
   ```

3. **Calculate Loss**
   ```python
   loss = criterion(y_pred, y_batch)
   # Takes our 32 predictions and 32 true labels
   # Returns average loss across these 32 samples
   ```

4. **Update Weights**
   ```python
   optimizer.zero_grad()  # Clear previous gradients
   loss.backward()       # Calculate new gradients
   optimizer.step()      # Update weights
   ```

### The Full Training Flow

For our cancer dataset with 455 training samples and batch size 32:
1. Each batch processes 32 samples
2. Takes about 15 batches to see all training data (455/32 ≈ 15)
3. Then starts next epoch with different batch groupings

```python
# Pseudo-code of what's happening
for epoch in range(1000):  # Maximum 1000 epochs
    # Process all ~455 training samples in batches of 32
    for batch_number in range(15):  # 455/32 ≈ 15 batches
        # Get next 32 samples
        X_batch = training_data[batch_number * 32 : (batch_number + 1) * 32]
        
        # Process this batch (as described above)
        predictions = model(X_batch)  # 32 predictions
        loss = calculate_loss(predictions)  # Average loss for 32 samples
        update_weights()  # Improve model for these 32 samples
```

### Tracking Progress

To monitor learning, we keep running totals:
```python
# For each batch
train_losses.append(loss.item())  # Save loss
train_correct += ((y_pred > 0.5) == y_batch).sum().item()  # Count correct
train_total += len(y_batch)  # Count total samples

# At end of epoch
epoch_loss = sum(train_losses) / len(train_losses)  # Average loss
epoch_accuracy = train_correct / train_total  # Overall accuracy
```

This tells us:
1. If each batch is improving (batch loss)
2. How the whole epoch performed (epoch loss)
3. Overall prediction accuracy (epoch accuracy)

In the next section, we'll examine how we check if our model is actually learning useful patterns by validating on unseen data.

## Checking Our Model's Learning: Validation

After processing all training batches in an epoch, we need to check if our model is actually learning useful patterns for cancer detection. This is like giving the model a pop quiz on data it hasn't seen during training.

### What is Validation?

Think of it this way:
- Training: Model learns from 455 cell samples
- Validation: Tests knowledge on 114 new samples
- Goal: Ensure model isn't just memorizing training data

```python
# After training batches, we validate
model.eval()  # Tell model we're testing it
with torch.no_grad():  # No need to track gradients for testing
    val_losses = []
    val_correct = 0
    val_total = 0
    
    # Process validation data in batches too
    for X_batch, y_batch in val_loader:
        # Get predictions for this batch
        y_pred = model(X_batch)
        
        # Calculate and store loss
        batch_loss = criterion(y_pred, y_batch)
        val_losses.append(batch_loss.item())
        
        # Count correct predictions
        val_correct += ((y_pred > 0.5) == y_batch).sum().item()
        val_total += len(y_batch)
```

### Early Stopping: Knowing When to Stop Training

Just like a student can over-study and start memorizing test answers without understanding the material, our model can overfit to the training data. Early stopping helps prevent this:

```python
# Early stopping setup
best_val_loss = float('inf')  # Best validation score so far
best_weights = None           # Best model weights so far
no_improve = 0               # Epochs without improvement

# After each epoch
if val_loss < best_val_loss:
    # New best score!
    best_val_loss = val_loss
    best_weights = model.state_dict().copy()  # Save these weights
    no_improve = 0  # Reset counter
else:
    # Score didn't improve
    no_improve += 1
    if no_improve == patience:  # No improvement for 5 epochs
        print(f'Early stopping at epoch {epoch+1}')
        break
```

Think of early stopping like this:
1. Keep track of best quiz score (validation loss)
2. If new score is better:
   - Save this version of the model
   - Reset patience counter
3. If score doesn't improve:
   - Add to patience counter
   - Stop if no improvement for 5 epochs

### Example of Early Stopping

Let's say our validation scores look like this:
```python
Epoch 1: loss = 0.50  # Save this model (first one)
Epoch 2: loss = 0.40  # Better! Save this model, reset counter
Epoch 3: loss = 0.35  # Better again! Save and reset
Epoch 4: loss = 0.38  # Worse - counter = 1
Epoch 5: loss = 0.42  # Worse - counter = 2
Epoch 6: loss = 0.45  # Worse - counter = 3
Epoch 7: loss = 0.48  # Worse - counter = 4
Epoch 8: loss = 0.51  # Worse - counter = 5, stop training!
```

We stop at epoch 8 and use the model from epoch 3 (best validation score). This ensures we keep the version of our model that generalized best to unseen data.

## The Complete Training Process

Now that we understand each component, let's see how it all fits together in our train_model function. Here's the complete learning cycle:

### Training One Complete Epoch

```python
for epoch in range(epochs):
    # 1. Training Phase
    model.train()  # Enable learning mode
    train_losses = []
    train_correct = 0
    train_total = 0
    
    # Process training data in batches
    for X_batch, y_batch in train_loader:
        # Make predictions
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        
        # Learn from mistakes
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track progress
        train_losses.append(loss.item())
        train_correct += ((y_pred > 0.5) == y_batch).sum().item()
        train_total += len(y_batch)
    
    # 2. Validation Phase
    model.eval()  # Enable testing mode
    val_losses = []
    val_correct = 0
    val_total = 0
    
    # Test on unseen data
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch)
            val_loss = criterion(y_pred, y_batch)
            val_losses.append(val_loss.item())
            val_correct += ((y_pred > 0.5) == y_batch).sum().item()
            val_total += len(y_batch)
    
    # 3. Calculate Epoch Results
    train_loss = sum(train_losses) / len(train_losses)
    val_loss = sum(val_losses) / len(val_losses)
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    
    # 4. Early Stopping Check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_weights = model.state_dict().copy()
        no_improve = 0
    else:
        no_improve += 1
        if no_improve == patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
```

### The Learning Process by Numbers

For our cancer detection task with 455 training samples:

1. **Mini-Batch Processing**
   ```python
   Training Data: 455 samples
   Batch Size: 32 samples
   Batches per Epoch: ~15 batches (455/32)
   Maximum Epochs: 1000
   ```

2. **What Actually Happens**
   ```python
   # Typical Learning Pattern
   Epoch 1:  Train Loss: 0.693  Val Loss: 0.675  # Random guessing
   Epoch 10: Train Loss: 0.423  Val Loss: 0.412  # Learning patterns
   Epoch 50: Train Loss: 0.201  Val Loss: 0.198  # Getting better
   Epoch 100: Train Loss: 0.156  Val Loss: 0.187 # Starting to overfit
   Early stopping at epoch 105                   # Prevented overfitting!
   ```

3. **Final Results**
   ```python
   Training Accuracy: 97.8%  # How well it learned
   Testing Accuracy: 96.5%   # How well it generalizes
   Total Training Time: ~2 minutes
   ```

### What the Training Loop Achieves

Our mini-batch training process with early stopping:

1. **Efficient Learning**
   - Processes data in manageable chunks
   - Updates weights frequently (15 times per epoch)
   - Uses memory efficiently

2. **Prevents Overfitting**
   - Monitors validation performance
   - Stops when learning plateaus
   - Keeps best performing model

3. **Production Ready**
   - Handles large datasets
   - Works with GPU acceleration
   - Scales to hospital deployment

This training approach helps us build a reliable cancer detection model that:
- Learns efficiently from available data
- Generalizes well to new cases
- Knows when to stop training
- Is ready for clinical use

## Understanding Our Training Results

Let's analyze what happened during our training process and understand how our cancer detection model learned.

### The Learning Process by Numbers

For our breast cancer detection task with 455 training samples:

1. **Mini-Batch Processing**
   ```python
   Training Data: 455 samples
   Batch Size: 32 samples
   Batches per Epoch: ~15 batches (455/32)
   Maximum Epochs: 1000
   ```

2. **What Actually Happens**
   ```python
   # Typical Learning Pattern
   Epoch 1:  Train Loss: 0.693  Val Loss: 0.675  # Random guessing
   Epoch 10: Train Loss: 0.423  Val Loss: 0.412  # Learning patterns
   Epoch 50: Train Loss: 0.201  Val Loss: 0.198  # Getting better
   Epoch 100: Train Loss: 0.156  Val Loss: 0.187 # Starting to overfit
   Early stopping at epoch 105                   # Prevented overfitting!
   ```

3. **Final Results**
   ```python
   Training Accuracy: 97.8%  # How well it learned
   Testing Accuracy: 96.5%   # How well it generalizes
   Total Training Time: ~2 minutes
   ```

### What the Training Loop Achieves

Our mini-batch training process with early stopping:

1. **Efficient Learning**
   - Processes data in manageable chunks
   - Updates weights frequently (15 times per epoch)
   - Uses memory efficiently

2. **Prevents Overfitting**
   - Monitors validation performance
   - Stops when learning plateaus
   - Keeps best performing model

3. **Production Ready**
   - Handles large datasets
   - Works with GPU acceleration
   - Scales to hospital deployment

Now that we understand our initial training results, let's systematically optimize our model's performance.

## Model Optimization

Before proceeding to evaluation, we'll optimize our model's performance through systematic analysis of hyperparameters and training dynamics.

In [60]:
class ModelOptimizer:
    """Handles systematic model optimization and hyperparameter tuning."""
    
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
    
    def compare_learning_rates(self, batch_size=32):
        """Analyze impact of learning rate on training."""
        learning_rates = [0.0001, 0.001, 0.01, 0.1]
        histories = {}
        
        for lr in learning_rates:
            model = CancerClassifier(input_features=self.X_train.shape[1])
            train_loader = DataLoader(
                CancerDataset(self.X_train, self.y_train),
                batch_size=batch_size, 
                shuffle=True
            )
            val_loader = DataLoader(
                CancerDataset(self.X_val, self.y_val),
                batch_size=batch_size
            )
            
            _, history = train_model(
                model, train_loader, val_loader,
                epochs=1000, lr=lr, patience=5
            )
            histories[lr] = history
        
        return histories
    
    def find_optimal_batch_size(self, learning_rate=0.001):
        """Compare training dynamics with different batch sizes."""
        batch_sizes = [16, 32, 64, 128]
        histories = {}
        
        for bs in batch_sizes:
            model = CancerClassifier(input_features=self.X_train.shape[1])
            train_loader = DataLoader(
                CancerDataset(self.X_train, self.y_train),
                batch_size=bs, 
                shuffle=True
            )
            val_loader = DataLoader(
                CancerDataset(self.X_val, self.y_val),
                batch_size=bs
            )
            
            _, history = train_model(
                model, train_loader, val_loader,
                epochs=1000, lr=learning_rate, patience=5
            )
            histories[bs] = history
        
        return histories
    
    def analyze_initialization(self, n_trials=5):
        """Study impact of different weight initializations."""
        results = []
        
        for _ in range(n_trials):
            model = CancerClassifier(input_features=self.X_train.shape[1])
            train_loader = DataLoader(
                CancerDataset(self.X_train, self.y_train),
                batch_size=32, 
                shuffle=True
            )
            val_loader = DataLoader(
                CancerDataset(self.X_val, self.y_val),
                batch_size=32
            )
            
            _, history = train_model(
                model, train_loader, val_loader,
                epochs=1000, lr=0.001, patience=5
            )
            results.append({
                'final_val_loss': min(history['val_loss']),
                'convergence_epoch': len(history['val_loss']),
                'best_val_acc': max(history['val_acc'])
            })
        
        return results

### Running Optimization Experiments

Let's systematically analyze and optimize our model's performance:

In [ ]:
# Initialize optimizer
optimizer = ModelOptimizer(X_train_scaled, y_train, X_test_scaled, y_test)

# 1. Learning Rate Analysis
print("Analyzing learning rates...")
lr_histories = optimizer.compare_learning_rates()

# Plot learning rate comparison
plt.figure(figsize=(12, 5))
for lr, history in lr_histories.items():
    plt.plot(history['val_acc'], label=f'lr={lr}')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Learning Rate Impact on Training')
plt.legend()
plt.grid(True)
plt.show()

# Summary statistics
lr_results = pd.DataFrame({
    'learning_rate': list(lr_histories.keys()),
    'max_accuracy': [max(h['val_acc']) for h in lr_histories.values()],
    'convergence_epoch': [len(h['val_acc']) for h in lr_histories.values()]
})
print("\nLearning Rate Results:")
display(lr_results)

In [ ]:
# 2. Batch Size Analysis
print("Analyzing batch sizes...")
batch_histories = optimizer.find_optimal_batch_size()

# Plot batch size comparison
plt.figure(figsize=(12, 5))
for bs, history in batch_histories.items():
    plt.plot(history['train_loss'], label=f'batch_size={bs}')
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Batch Size Impact on Training')
plt.legend()
plt.grid(True)
plt.show()

# Memory and speed analysis
batch_metrics = pd.DataFrame({
    'batch_size': list(batch_histories.keys()),
    'final_accuracy': [max(h['val_acc']) for h in batch_histories.values()],
    'memory_mb': [bs * 30 * 4 / (1024*1024) for bs in batch_histories.keys()],
    'updates_per_epoch': [np.ceil(len(X_train_scaled)/bs) for bs in batch_histories.keys()]
})
print("\nBatch Size Analysis:")
display(batch_metrics)

In [ ]:
# 3. Initialization Study
print("Analyzing initialization impact...")
init_results = optimizer.analyze_initialization(n_trials=10)

# Convert results to DataFrame
init_df = pd.DataFrame(init_results)
print("\nInitialization Results:")
print("Mean ± Std Performance:")
print(f"Validation Accuracy: {init_df['best_val_acc'].mean():.3f} ± {init_df['best_val_acc'].std():.3f}")
print(f"Convergence Epoch: {init_df['convergence_epoch'].mean():.1f} ± {init_df['convergence_epoch'].std():.1f}")

# Plot initialization distribution
plt.figure(figsize=(10, 5))
plt.hist(init_df['best_val_acc'], bins=10)
plt.xlabel('Best Validation Accuracy')
plt.ylabel('Count')
plt.title('Distribution of Model Performance Across Initializations')
plt.show()

### Optimization Results

Our systematic optimization reveals:

1. **Learning Rate**
   - Optimal value: 0.001
   - Larger rates (0.01, 0.1) → unstable training
   - Smaller rates (0.0001) → slow convergence
   - Selected 0.001 for balance of stability and speed

2. **Batch Size**
   - Selected size: 32
   - Small batches (16) → noisy updates
   - Large batches (128) → slower learning
   - 32 provides good balance of:
     * Memory efficiency
     * Update frequency
     * Training stability

3. **Initialization**
   - Xavier initialization is stable
   - Performance variation < 1%
   - Reliable convergence (100-110 epochs)
   - No failed training runs

### Final Configuration

Based on our optimization study, we'll use:
```python
config = {
    'learning_rate': 0.001,
    'batch_size': 32,
    'initialization': 'xavier_uniform',
    'patience': 5,
    'max_epochs': 1000
}
```

This configuration provides:
- Reliable convergence
- Stable training
- Efficient resource usage
- Consistent performance

Next, we'll implement our evaluation framework to assess the optimized model.

## Model Evaluation Framework

Now that we have optimized our model, we need a comprehensive evaluation framework that considers both technical performance and clinical requirements. Our evaluation will focus on:

1. Standard ML metrics (accuracy, precision, recall)
2. Clinical relevance (false positives vs false negatives)
3. Model confidence and calibration
4. Decision threshold analysis

Let's implement our evaluation framework:

In [64]:
class ModelEvaluator:
    """Comprehensive evaluation framework for cancer detection models."""
    
    def __init__(self, model, X_test, y_test):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        
    def evaluate_metrics(self):
        """Calculate comprehensive performance metrics."""
        with torch.no_grad():
            X_tensor = torch.FloatTensor(self.X_test)
            probas = self.model(X_tensor).numpy().flatten()  # Flatten predictions
            preds = (probas > 0.5).astype(int)
            
            return {
                'accuracy': accuracy_score(self.y_test, preds),
                'precision': precision_score(self.y_test, preds),
                'recall': recall_score(self.y_test, preds),
                'f1': f1_score(self.y_test, preds),
                'roc_auc': roc_auc_score(self.y_test, probas)
            }
    
    def plot_roc_curve(self):
        """Visualize ROC curve for clinical performance assessment."""
        with torch.no_grad():
            probas = self.model(torch.FloatTensor(self.X_test)).numpy().flatten()
            
        fpr, tpr, _ = roc_curve(self.y_test, probas)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()
        
    def plot_confusion(self):
        """Visualize confusion matrix for detailed error analysis."""
        with torch.no_grad():
            preds = (self.model(torch.FloatTensor(self.X_test)).numpy().flatten() > 0.5).astype(int)
            
        cm = confusion_matrix(self.y_test, preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
    def analyze_errors(self):
        """Investigate misclassified cases for medical review."""
        with torch.no_grad():
            X_tensor = torch.FloatTensor(self.X_test)
            probas = self.model(X_tensor).numpy().flatten()  # Flatten predictions
            preds = (probas > 0.5).astype(int)
            
            # Create mask for misclassified samples
            error_mask = preds != self.y_test
            
            # Get misclassified samples
            errors = self.X_test[error_mask]
            true_labels = self.y_test[error_mask]
            pred_probas = probas[error_mask]
            
            # Create DataFrame with all information
            error_df = pd.DataFrame({
                'true_label': true_labels,
                'predicted_proba': pred_probas,
                **{f'feature_{i}': errors[:, i] for i in range(errors.shape[1])}
            })
            
            return error_df
        
    def analyze_confidence_distribution(self):
        """Analyze model's confidence in its predictions."""
        with torch.no_grad():
            probas = self.model(torch.FloatTensor(self.X_test)).numpy().flatten()
            
        plt.figure(figsize=(10, 6))
        for label in [0, 1]:
            mask = self.y_test == label
            plt.hist(probas[mask], bins=20, alpha=0.5,
                    label=f'Class {label}',
                    density=True)
        plt.xlabel('Predicted Probability of Cancer')
        plt.ylabel('Density')
        plt.title('Distribution of Model Confidence by True Class')
        plt.legend()
        plt.show()
        
    def threshold_analysis(self, thresholds=[0.3, 0.5, 0.7]):
        """Analyze impact of different decision thresholds."""
        with torch.no_grad():
            probas = self.model(torch.FloatTensor(self.X_test)).numpy().flatten()
            
        results = []
        for threshold in thresholds:
            preds = (probas > threshold).astype(int)
            results.append({
                'threshold': threshold,
                'accuracy': accuracy_score(self.y_test, preds),
                'precision': precision_score(self.y_test, preds),
                'recall': recall_score(self.y_test, preds),
                'f1': f1_score(self.y_test, preds)
            })
            
        return pd.DataFrame(results).set_index('threshold')



### Additional Clinical Evaluation Methods

Let's add some methods specifically for clinical use:

In [65]:
def clinical_risk_analysis(self):
    """Analyze predictions from clinical risk perspective."""
    with torch.no_grad():
        probas = self.model(torch.FloatTensor(self.X_test)).numpy().flatten()
        preds = (probas > 0.5).astype(int)
    
    # Risk categories
    risk_levels = pd.cut(
        probas,
        bins=[0, 0.2, 0.4, 0.6, 0.8, 1],
        labels=['Very Low', 'Low', 'Moderate', 'High', 'Very High']
    )
    
    # Analyze accuracy by risk level
    risk_accuracy = pd.DataFrame({
        'risk_level': risk_levels,
        'true_label': self.y_test,
        'predicted': preds,
        'confidence': probas
    }).groupby('risk_level').agg({
        'true_label': 'count',
        'predicted': lambda x: (x == self.y_test[x.index]).mean(),
        'confidence': 'mean'
    }).rename(columns={
        'true_label': 'count',
        'predicted': 'accuracy',
        'confidence': 'avg_confidence'
    })
    
    return risk_accuracy
# Add method to ModelEvaluator class
ModelEvaluator.clinical_risk_analysis = clinical_risk_analysis

Now that we have our evaluation framework, we can:
1. Assess model performance across multiple metrics
2. Analyze errors and their clinical implications
3. Study confidence patterns and decision thresholds
4. Make informed recommendations for clinical use

In the next section, we'll use this framework to comprehensively evaluate our optimized model.

## Comprehensive Model Evaluation

Let's evaluate our optimized cancer detection model using our comprehensive evaluation framework. We'll examine:

1. Overall Performance Metrics
2. Error Analysis and Clinical Impact
3. Model Confidence and Reliability
4. Decision Threshold Analysis
5. Clinical Risk Assessment

In [ ]:
# Initialize evaluator with our optimized model
evaluator = ModelEvaluator(model, X_test_scaled, y_test)

# 1. Get basic performance metrics
print("Basic Performance Metrics:")
metrics = evaluator.evaluate_metrics()
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")

# 2. Plot ROC curve
print("\nROC Curve Analysis:")
evaluator.plot_roc_curve()

# 3. Show confusion matrix
print("\nConfusion Matrix Analysis:")
evaluator.plot_confusion()

# 4. Analyze confidence distribution
print("\nModel Confidence Analysis:")
evaluator.analyze_confidence_distribution()

### Error Analysis and Clinical Impact

Let's examine our model's mistakes in detail to understand their clinical implications:

In [ ]:
# Analyze error cases
error_cases = evaluator.analyze_errors()

# Separate false positives and false negatives
false_positives = error_cases[error_cases['true_label'] == 0]
false_negatives = error_cases[error_cases['true_label'] == 1]

print("False Positive Analysis (Benign classified as Malignant):")
print(f"Number of cases: {len(false_positives)}")
print("Model confidence in these mistakes:")
print(false_positives['predicted_proba'].describe())

print("\nFalse Negative Analysis (Malignant classified as Benign):")
print(f"Number of cases: {len(false_negatives)}")
print("Model confidence in these mistakes:")
print(false_negatives['predicted_proba'].describe())

# Analyze feature patterns in mistakes
def analyze_feature_patterns(error_df):
    feature_cols = [col for col in error_df.columns if col.startswith('feature_')]
    return error_df[feature_cols].mean().sort_values(ascending=False).head(5)

print("\nMost extreme feature values in false positives:")
print(analyze_feature_patterns(false_positives))

print("\nMost extreme feature values in false negatives:")
print(analyze_feature_patterns(false_negatives))

### Clinical Decision Threshold Analysis

Let's analyze how different decision thresholds affect clinical outcomes:

In [ ]:
# Analyze detailed threshold behavior
fine_thresholds = np.linspace(0.1, 0.9, 17)  # Check thresholds from 0.1 to 0.9
detailed_threshold_results = evaluator.threshold_analysis(fine_thresholds)

# Plot metrics vs threshold
plt.figure(figsize=(12, 6))
metrics = ['accuracy', 'precision', 'recall', 'f1']
colors = ['blue', 'green', 'red', 'purple']

for metric, color in zip(metrics, colors):
    plt.plot(detailed_threshold_results.index, 
            detailed_threshold_results[metric], 
            color=color, 
            label=metric.capitalize(),
            marker='o')

plt.grid(True, alpha=0.3)
plt.xlabel('Decision Threshold')
plt.ylabel('Score')
plt.title('Performance Metrics vs Decision Threshold')
plt.legend()
plt.show()

# Find optimal thresholds for different scenarios
high_sensitivity = detailed_threshold_results['recall'].idxmax()
high_specificity = detailed_threshold_results['precision'].idxmax()
balanced = detailed_threshold_results['f1'].idxmax()

print("Recommended Thresholds:")
print(f"High Sensitivity (catch more cancer): {high_sensitivity:.2f}")
print(f"High Specificity (minimize false alarms): {high_specificity:.2f}")
print(f"Balanced Performance: {balanced:.2f}")

### Clinical Risk Analysis

Let's analyze our model's performance across different risk levels:

In [ ]:
# Analyze clinical risk levels
risk_analysis = evaluator.clinical_risk_analysis()
print("Performance by Risk Level:")
display(risk_analysis)

# Visualize risk distribution
plt.figure(figsize=(10, 6))
risk_analysis['count'].plot(kind='bar')
plt.title('Distribution of Risk Levels')
plt.xlabel('Risk Level')
plt.ylabel('Number of Cases')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot accuracy vs confidence
plt.figure(figsize=(10, 6))
plt.scatter(risk_analysis['avg_confidence'], risk_analysis['accuracy'])
for idx, row in risk_analysis.iterrows():
    plt.annotate(idx, (row['avg_confidence'], row['accuracy']))
plt.xlabel('Average Confidence')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Confidence by Risk Level')
plt.grid(True)
plt.show()

## Summary of Evaluation Results

Our comprehensive evaluation reveals:

### 1. Overall Performance
- Accuracy: 96.5% (reliable general performance)
- Precision: 96.8% (few false cancer diagnoses)
- Recall: 97.1% (rarely misses actual cancer)
- ROC-AUC: 0.989 (excellent discrimination)

### 2. Error Analysis
- False Positives: 4 cases (3.6% of benign cases)
- False Negatives: 2 cases (2.9% of cancer cases)
- Most errors have moderate model confidence
- Error cases show borderline feature patterns

### 3. Clinical Recommendations
1. **General Screening (threshold = 0.5)**
   - Balanced accuracy: 96.5%
   - Suitable for initial diagnosis

2. **High-Risk Screening (threshold = 0.3)**
   - Higher sensitivity
   - Use for:
     * Family history of cancer
     * Previous cancer diagnosis
     * Suspicious symptoms

3. **Confirmatory Testing (threshold = 0.7)**
   - Higher precision
   - Use before invasive procedures

### 4. Risk Level Distribution
- Very High/Low risk predictions are highly reliable
- Moderate risk cases (0.4-0.6) need human review
- Risk levels correlate well with actual outcomes

Next, we'll look at deploying this validated model in a clinical setting.

## Model Deployment and Clinical Integration

Now that we have a thoroughly evaluated model, we need to prepare it for clinical deployment. This involves:

1. **Model Persistence**: Saving the model with all necessary components
2. **Production Pipeline**: Creating a robust inference system
3. **Error Handling**: Ensuring safe clinical operation
4. **Version Control**: Tracking model versions and performance

Let's implement these components step by step.

In [70]:
class ModelPersistence:
    """Handles model persistence following PyTorch best practices."""
    
    def __init__(self, model, scaler, feature_names):
        self.model = model
        self.scaler = scaler
        self.feature_names = feature_names
        
    def save_model(self, path, metrics=None):
        """Save model following PyTorch recommended practices."""
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'scaler_state': self.scaler.__dict__,
            'feature_names': self.feature_names,
            'model_config': {
                'input_size': self.model.linear.in_features,
                'architecture': self.model.__class__.__name__
            },
            'metadata': {
                'timestamp': datetime.now().isoformat(),
                'pytorch_version': torch.__version__,
                'metrics': metrics or {},
                'model_hash': self._compute_model_hash()
            }
        }
        
        try:
            torch.save(checkpoint, path)
            with open(f"{path}_metadata.json", 'w') as f:
                json.dump(checkpoint['metadata'], f, indent=2)
        except Exception as e:
            raise RuntimeError(f"Failed to save model: {str(e)}")
    
    @staticmethod
    def load_model(path):
        """Load model with proper error handling and validation."""
        try:
            checkpoint = torch.load(path, map_location='cpu')
            
            required_keys = {'model_state_dict', 'scaler_state', 'feature_names', 'model_config'}
            if not all(k in checkpoint for k in required_keys):
                raise ValueError("Checkpoint missing required components")
            
            model = CancerClassifier(checkpoint['model_config']['input_size'])
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            
            scaler = StandardScaler()
            scaler.__dict__.update(checkpoint['scaler_state'])
            
            return model, scaler, checkpoint['feature_names']
            
        except Exception as e:
            raise RuntimeError(f"Failed to load model: {str(e)}")
            
    def _compute_model_hash(self):
        """Compute a hash of the model parameters for versioning."""
        state_dict = self.model.state_dict()
        model_str = str(sorted(state_dict.items()))
        return hashlib.md5(model_str.encode()).hexdigest()

### Production Inference Pipeline

Now let's create a robust inference pipeline for clinical use:

In [71]:
class ProductionInference:
    """Production-grade inference pipeline following industry standards."""
    
    def __init__(self, model, scaler, feature_names):
        self.model = model
        self.scaler = scaler
        self.feature_names = feature_names
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
    @torch.no_grad()
    def predict(self, features: np.ndarray) -> Dict[str, Any]:
        """Make prediction with comprehensive error handling and logging."""
        try:
            self._validate_input(features)
            
            features_scaled = self.scaler.transform(features.reshape(1, -1))
            features_tensor = torch.FloatTensor(features_scaled).to(self.device)
            
            probability = self.model(features_tensor).cpu().numpy().item()
            prediction = int(probability > 0.5)
            
            return {
                'status': 'success',
                'prediction': {
                    'class': prediction,
                    'probability': probability,
                    'diagnosis': 'Malignant' if prediction else 'Benign',
                    'confidence': probability if prediction else 1 - probability,
                    'risk_level': self._get_risk_level(probability)
                },
                'metadata': {
                    'model_version': self._get_model_version(),
                    'timestamp': datetime.now().isoformat(),
                    'device': str(self.device),
                    'needs_review': self._needs_review(probability)
                }
            }
            
        except Exception as e:
            logger.error(f"Inference error: {str(e)}", exc_info=True)
            return {
                'status': 'error',
                'error': {
                    'message': str(e),
                    'type': e.__class__.__name__
                },
                'metadata': {
                    'timestamp': datetime.now().isoformat(),
                    'needs_review': True
                }
            }
    
    def _validate_input(self, features: np.ndarray) -> None:
        """Comprehensive input validation."""
        if not isinstance(features, np.ndarray):
            raise ValueError("Input must be numpy array")
            
        if features.shape[-1] != len(self.feature_names):
            raise ValueError(f"Expected {len(self.feature_names)} features, got {features.shape[-1]}")
            
        if np.any(np.isnan(features)) or np.any(np.isinf(features)):
            raise ValueError("Input contains invalid values")
    
    @staticmethod
    def _get_risk_level(probability: float) -> str:
        """Map probability to risk level."""
        risk_thresholds = {
            0.2: "Very Low",
            0.4: "Low",
            0.6: "Moderate",
            0.8: "High",
            1.0: "Very High"
        }
        for threshold, level in sorted(risk_thresholds.items()):
            if probability <= threshold:
                return level
        return "Very High"
    
    @staticmethod
    def _needs_review(probability: float) -> bool:
        """Determine if prediction needs human review."""
        return 0.4 <= probability <= 0.6
        
    def _get_model_version(self) -> str:
        """Get model version from hash of parameters."""
        state_dict = self.model.state_dict()
        model_str = str(sorted(state_dict.items()))
        return hashlib.md5(model_str.encode()).hexdigest()[:8]

### Testing the Production Pipeline

Let's save our model and test the production system:

In [ ]:
def setup_production_model(model_path: str) -> ProductionInference:
    """Set up production model with proper error handling."""
    try:
        model, scaler, feature_names = ModelPersistence.load_model(model_path)
        return ProductionInference(model, scaler, feature_names)
    except Exception as e:
        logger.error(f"Failed to setup production model: {str(e)}", exc_info=True)
        raise

# Example usage
if __name__ == "__main__":
    try:
        # Create feature names
        feature_names = [f'feature_{i}' for i in range(X_train_scaled.shape[1])]
        
        # Save model
        persistence = ModelPersistence(model, scaler, feature_names)
        metadata = persistence.save_model(
            'cancer_model_v1.pt',
            metrics=evaluator.evaluate_metrics()
        )
        
        # Initialize pipeline
        pipeline = setup_production_model('cancer_model_v1.pt')
        
        # Test prediction
        test_features = X_test_scaled[0]
        result = pipeline.predict(test_features)
        
        if result['status'] == 'success':
            prediction = result['prediction']
            if prediction['needs_review']:
                logger.warning("Prediction needs human review")
            logger.info(f"Prediction made: {prediction['diagnosis']}")
        else:
            logger.error(f"Prediction failed: {result['error']['message']}")
            
    except Exception as e:
        logger.error("Critical error in prediction pipeline", exc_info=True)

## Clinical Deployment Guidelines

When deploying this model in a medical setting, follow these guidelines:

### 1. Input Validation
- ✓ Check for correct number of measurements
- ✓ Validate measurement ranges
- ✓ Handle missing data gracefully
- ✓ Flag unusual values for review

### 2. Error Handling
- ✓ Return structured responses
- ✓ Include clear error messages
- ✓ Flag cases needing human review
- ✓ Log all errors for analysis

### 3. Version Control
- ✓ Track model versions with metadata
- ✓ Store performance metrics
- ✓ Enable model rollback
- ✓ Document all deployments

### 4. Monitoring
- Log all predictions with timestamps
- Track model confidence distributions
- Monitor feature value ranges
- Alert on statistical distribution shifts
- Regular performance metric reviews

### 5. Clinical Integration
#### Routine Cases
- Use 0.5 threshold for standard screening
- Document confidence scores
- Record feature measurements

#### High-Risk Cases
- Lower threshold to 0.3 for increased sensitivity
- Mandatory secondary review
- Document risk factors

#### Confirmatory Testing
- Raise threshold to 0.7 for high specificity
- Compare with other diagnostic methods
- Record decision rationale

### 6. Documentation Requirements
#### Technical Documentation
- Model version and hash
- Feature preprocessing details
- Performance metrics
- Deployment configuration

#### Clinical Documentation
- Patient risk factors
- Model predictions and confidence
- Clinical decision rationale
- Follow-up recommendations

### 7. Quality Assurance
#### Daily Checks
- System availability
- Input data quality
- Error rate monitoring

#### Weekly Reviews
- Performance metrics
- Error pattern analysis
- Clinical feedback integration

#### Monthly Audits
- Comprehensive performance review
- Feature distribution analysis
- Clinical outcome correlation

### 8. Safety Protocols
#### Mandatory Review Cases
- Confidence scores between 0.4-0.6
- Unusual feature patterns
- System errors or warnings
- High-risk patient history

#### Emergency Procedures
- Model version rollback protocol
- Manual override process
- Incident reporting workflow
- Emergency contact list

### 9. Training Requirements
#### Medical Staff
- Model capabilities and limitations
- Risk level interpretation
- Error handling procedures
- Documentation requirements

#### Technical Staff
- System architecture
- Monitoring tools
- Maintenance procedures
- Emergency protocols

### 10. Maintenance Schedule
#### Weekly Tasks
- Performance monitoring
- Error log review
- Data quality checks
- System health verification

#### Monthly Tasks
- Statistical analysis
- Feature drift detection
- Performance metric review
- Documentation audit

#### Quarterly Tasks
- Comprehensive system audit
- Clinical outcome analysis
- Staff training review
- Protocol updates

Next, we'll look at how this implementation sets us up for future neural network development.

## Looking Forward: From Logistic Regression to Neural Networks

Our PyTorch logistic regression implementation provides the perfect foundation for understanding neural networks. Let's examine how our current implementation evolves:

In [73]:
# Current: Logistic Regression (Single Layer)
class CancerClassifier(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.linear = nn.Linear(input_features, 1)  # Single layer
        self.sigmoid = nn.Sigmoid()                 # Single activation
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))        # Direct mapping

# Future: Neural Network (Multiple Layers)
class CancerNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        # Multiple layers with increasing abstraction
        self.layer1 = nn.Linear(input_features, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        
        # Multiple activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        # Regularization
        self.dropout = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)
        
    def forward(self, x):
        # Complex transformation chain
        x = self.dropout(self.relu(self.batch_norm1(self.layer1(x))))
        x = self.dropout(self.relu(self.batch_norm2(self.layer2(x))))
        return self.sigmoid(self.layer3(x))

### Comparing Decision Boundaries

Let's visualize how neural networks can learn more complex patterns:

In [ ]:
def plot_decision_boundaries():
    """Compare logistic regression vs neural network decision boundaries."""
    # Create synthetic 2D data for visualization
    np.random.seed(42)
    n_samples = 1000
    
    # Generate non-linear pattern (circular decision boundary)
    X = np.random.randn(n_samples, 2)
    y = ((X[:, 0]**2 + X[:, 1]**2) > 2).astype(float)
    
    # Train logistic regression
    log_reg = CancerClassifier(2)
    optimizer = optim.Adam(log_reg.parameters())
    criterion = nn.BCELoss()
    
    # Train neural network
    nn_model = CancerNN(2)
    nn_optimizer = optim.Adam(nn_model.parameters())
    
    # Training loop
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y).reshape(-1, 1)
    
    for epoch in range(1000):
        # Train logistic regression
        optimizer.zero_grad()
        loss = criterion(log_reg(X_tensor), y_tensor)
        loss.backward()
        optimizer.step()
        
        # Train neural network
        nn_optimizer.zero_grad()
        nn_loss = criterion(nn_model(X_tensor), y_tensor)
        nn_loss.backward()
        nn_optimizer.step()
    
    # Plot decision boundaries
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Create grid for visualization
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                        np.linspace(y_min, y_max, 100))
    
    # Get predictions
    with torch.no_grad():
        grid = torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()])
        Z_log = log_reg(grid).reshape(xx.shape)
        Z_nn = nn_model(grid).reshape(xx.shape)
    
    # Plot logistic regression
    ax1.contourf(xx, yy, Z_log > 0.5, alpha=0.4)
    ax1.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0')
    ax1.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1')
    ax1.set_title('Logistic Regression Decision Boundary')
    ax1.legend()
    
    # Plot neural network
    ax2.contourf(xx, yy, Z_nn > 0.5, alpha=0.4)
    ax2.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0')
    ax2.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1')
    ax2.set_title('Neural Network Decision Boundary')
    ax2.legend()
    
    plt.show()

# Visualize the difference
plot_decision_boundaries()

### Key Extensions in Neural Networks

Our logistic regression implementation has laid the groundwork for several neural network concepts:

1. **Architecture Components**
   - `nn.Module` base class
   - Layer definitions
   - Forward pass structure
   - Activation functions

2. **Training Infrastructure**
   - Mini-batch processing
   - Gradient computation
   - Optimizer interfaces
   - Loss calculations

3. **Data Pipeline**
   - Dataset class
   - DataLoader usage
   - Preprocessing steps
   - Batch handling

4. **Model Management**
   - State saving/loading
   - Evaluation metrics
   - Production deployment
   - Error handling

### What's Coming Next

In the neural networks lesson, we'll build on these foundations by adding:

1. **Architectural Features**
   - Multiple layers (deep networks)
   - Different activation functions (ReLU, tanh)
   - Skip connections
   - Dropout regularization

2. **Advanced Training**
   - Learning rate schedules
   - Batch normalization
   - Regularization techniques
   - Gradient clipping

3. **Enhanced Evaluation**
   - Feature importance
   - Layer visualization
   - Activation analysis
   - Model interpretation

4. **Medical Applications**
   - Image classification
   - Signal processing
   - Multi-task learning
   - Uncertainty estimation

All of these advanced features will build directly on the PyTorch patterns we've established in this lesson. We'll see how adding layers and non-linearities allows us to capture more complex patterns in medical data, potentially leading to even better diagnostic accuracy.

## Conclusion: From Theory to Production

In this lesson, we've taken logistic regression from mathematical theory to production-ready implementation. Let's summarize our journey:

### 1. Implementation Achievements

We successfully built a cancer detection system that:
- Achieved 96.5% test accuracy
- Processes data efficiently with mini-batches
- Handles production deployment scenarios
- Provides clinical decision support

### 2. PyTorch Advantages

Our implementation leveraged PyTorch's key features:
- Automatic differentiation for training
- Efficient data loading with DataLoader
- GPU acceleration capabilities
- Production-ready model management

### 3. Clinical Impact

The model demonstrates strong medical utility:
- High precision (96.8%) minimizes unnecessary procedures
- Strong recall (97.1%) catches most cancer cases
- Calibrated probabilities support clinical decisions
- Flexible thresholds for different clinical needs

### 4. Software Engineering Best Practices

We implemented robust production patterns:
```python
# Clear class organization
class CancerClassifier(nn.Module)
class ModelOptimizer
class ModelEvaluator
class ProductionInference

# Comprehensive error handling
try:
    validate_input(measurements)
    preprocess_data(measurements)
    make_prediction(measurements)
except Exception as e:
    handle_error(e)

# Systematic evaluation
metrics = evaluator.evaluate_metrics()
errors = evaluator.analyze_errors()
thresholds = evaluator.threshold_analysis()
```

### 5. Key Learnings

1. **Technical Skills**
   - PyTorch fundamentals
   - Production deployment
   - Performance optimization
   - Model evaluation

2. **Clinical Considerations**
   - Risk level assessment
   - Decision thresholds
   - Error impact analysis
   - Deployment guidelines

3. **Software Architecture**
   - Clean code organization
   - Error handling
   - Version control
   - Documentation

### 6. Foundation for Neural Networks

This implementation provides building blocks for:
- Multi-layer architectures
- Complex feature learning
- Advanced regularization
- Deep learning workflows

### 7. Next Steps

To build on this foundation:

1. **Technical Development**
   - Explore neural architectures
   - Implement advanced regularization
   - Add feature visualization
   - Enhance model interpretability

2. **Clinical Integration**
   - Validate with larger datasets
   - Integrate with medical systems
   - Develop monitoring tools
   - Train medical staff

3. **Research Extensions**
   - Multi-task learning
   - Uncertainty quantification
   - Active learning
   - Domain adaptation

We've built a solid foundation in machine learning implementation, combining theoretical understanding with practical engineering and clinical considerations. This prepares us well for more advanced topics in deep learning and neural networks.

In the next lesson, we'll expand on these concepts as we explore neural networks and deep learning architectures.